<a href="https://colab.research.google.com/github/cmucung/eye-retina-disease-classification/blob/main/CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lesson 3 - Introduction to CNNs
===============================

@AIS 2023 (adapted from @AI4ALL 2022 code)

TA: Priyanshi

In this notebooke, we use Tensorflow/Keras library to play with CNNs.

Keras Documentation: https://keras.io/

Section Overview
- ✅ Importing Dependencies
- ✅ Visualizing an Image through CNN
- ✅ Loading MNIST dataset
- ✅ Constructing a CNN
- ✅ Training a CNN
- ✅ Playground

Importing Dependencies
-----------------------

In [1]:
!git clone -q https://github.com/Srinivas-R/AI4ALL.git

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy import signal
import requests
import io
import os
from urllib.request import urlopen
from PIL import Image
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

Part 1: Using Pre-Trained Models
------------------------------

We use a pretrained `VGG16` CNN.

In [2]:
# LOAD PRETRAINED VGG16
base_model = VGG16(weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 1s 0us/step


Part 2: Build Your Own Model:
----------------------



###Loading your Dataset

Load and visualize your dataset.

In [3]:

from google.colab import drive
drive.mount('/content/drive')
# EXERCISE 1 : LOAD YOUR OWN DATASET HERE

directory= "/content/drive/MyDrive/Group 4/Retina Classification/Dataset/archive/Training_Set/Training_Set/Training"
training_files = [file for file in os.listdir(directory) if file.endswith('.png')]
image_data = []

for file in training_files:
    file_path = os.path.join(directory, file)
    with Image.open(file_path) as img:
        width, height = img.size

    image_data.append({
        'File Name': file,
        'Width': width,
        'Height': height,
        # Add more features when we need
    })

df = pd.DataFrame(image_data)
print(df.head())

num_classes = 2


Mounted at /content/drive
  File Name  Width  Height
0  1024.png   2144    1424
1  1018.png   2144    1424
2  1019.png   2144    1424
3  1025.png   2144    1424
4  1031.png   2144    1424


Visualize the dataset.

In [4]:
rand_idx = np.random.randint(0, len(x_train))
# EXERCISE 3: VISUALISE AN IMAGE FROM YOUR DATASET
input_shape = (x_train.shape[1], x_train.shape[2], 1)
print('Input Shape: ', input_shape)
print('Label: ', y_train[rand_idx])


example = x_train[rand_idx, :, :]
plt.imshow(example, cmap='gray')

NameError: name 'x_train' is not defined

###Constructing a CNN

Construct a CNN with keras.Sequential()

In [ ]:
model = keras.Sequential() #Step #1: Define the model

_, height, width = x_train.shape
depth = 1 # 1 for grayscale images
input_shape = (height, width, depth)

print(f"Input Shape:", input_shape)
#Step 2: Add layers to the model:

# BLOCK 1
model.add(layers.Conv2D(filters=8, kernel_size=(3, 3), padding="same", activation="relu", input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# BLOCK 2
model.add(layers.Conv2D(filters=8, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# CLASSIFIER
model.add(layers.MaxPooling2D(pool_size=(7, 7)))
model.add(layers.Flatten())
model.add(layers.Dense(num_classes))
model.add(layers.Activation("softmax"))

model.summary()

Question: Can you figure out the logic for computing number of params ?

###Training a CNN

Define hyperparameters and train the CNN.

Define hyperparameters.

In [ ]:
# HYPERPARAMETRS

# num_epochs - number of times to repeat the training
num_epochs = 10

# batch_size - how many images to train together at each step
batch_size = 128

# learning_rate - the update speed of the network
learning_rate = 1e-3

Compile and start training.

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy']) #Step 3: Compile

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=num_epochs,
          verbose=1) #Step 4: Train the model



###Run Inference on New Data


In [ ]:
img_index = 23 #pick an image to test on from our test array, or try selecting it randomly like we did for visualization!

print(f"Running Model Inference on Image number {img_index}")
img_array = x_test[img_index]
expanded_img_array = np.expand_dims(img_array, axis=(0, 3))

actual_label = y_test[img_index]
actual_label = np.argmax(actual_label)
# Make predictions using the model
predictions = model.predict(expanded_img_array)
predicted_label = np.argmax(predictions[0])  # Get the predicted label

print("Predicted Label:", predicted_label)
print("Actual Label:", actual_label)


###Cross-Validation

We can use the entirety of the test arrays to compute our model's accuracy and loss on new data to see how it performs.

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0) #Step 5: Cross Validation
print('Test loss:', score[0])
print('Test accuracy:', score[1])

For any model, our aim is to minimize the "loss" and maximize the "accuracy". Here accuracy is the cross-validation probability of the event
(predicted label = actual label).

###Saving and Loading the model

We can also save the model for future use.

In [ ]:
# Save the model and its weights
model.save("path_to_save_model.h5")

# Load the saved model
loaded_model = keras.models.load_model("/content/path_to_save_model.h5")

Playground
----------
Try new ideas and play with CNN architecture.

How can we improve model accuracy? Messs around with model architecture and hyperparams to see how it impacts model performance. Here are some ideas

###Conv2D arguments

- Increase Conv2D channels (aka filters)?
- Kernel Size: Adjust the kernel_size parameter in the Conv2D layer to change the size of the convolutional filter. Experiment with different kernel sizes (e.g., 3x3, 5x5) to observe the impact on the model's ability to capture different spatial features.

###Layers:

- Increase the depth: Add more Conv2D layers to the network to increase its depth. You can stack additional convolutional layers to allow the model to learn more complex features and representations.

- Modify the pooling layers: Experiment with adding more MaxPooling2D layers or changing their sizes to alter the downsampling process. This can impact the spatial resolution of the feature maps and the amount of information preserved.

- Change the Dense layers: Add more Dense layers in the classifier part of the network to increase its capacity. You can also experiment with different activation functions in these layers to observe their effect on the model's performance.

###Training Hyperparameters

- Activation Function: Try using different activation functions in the Conv2D and Dense layers, such as ReLU, sigmoid, or tanh. Each activation function has different properties that can impact the model's ability to learn and converge.
- Learning Rate: Experiment with different learning rates to observe their effect on the training process. Higher learning rates may result in faster convergence but may also introduce instability, while lower learning rates may slow down convergence but provide more stable learning.
- Number of Epochs: Increase or decrease the number of training epochs to see how it affects the model's performance. More epochs allow the model to learn for longer, but there's a risk of overfitting if the model is trained for too long.
- Batch Size: Try varying the batch size used during training. Larger batch sizes may accelerate training due to more efficient parallelization, while smaller batch sizes can offer better generalization at the cost of longer training time.

Please record what you have tried and what's the impact here.